**Imports**

In [1]:

import os
import pandas as pd

import sklearn as sk
import sklearn.feature_selection
import sklearn.pipeline

import pyplotterlib.standard.plotters as ppl

#
import project_path
import preproc_pipes as preProcPipeHelp

**Configuration Variables**

In [2]:
TRAIN_PATH = os.path.join("..","..","raw_data","train.csv")

#Want to restrict to recent data only
N_DAYS_USE = 400
# N_DAYS_USE = 50


#Lag values for sales and number of items on promotion
SALE_LAG_VALS = [x for x in range(16,30)]
ON_PROM_LAGS = [1,2,3,4,5,6,7,14,15,16,17,18]

#The obvious lag here is 0; since we actually always KNOW todays (and even future days)
#(so we use lag_0 with these moving average windows)
ON_PROM_WINDOW_VALS = [2,3,4,5,6,7,14,21]

#Obvious lag is 16; since we only need to predict 15 days into the future 
#So these moving averages at sakes lag=16
SALE_WINDOW_VALS = [2,3,4,5,6,7,14,21]

#
_currKwargs = {"constrainedLayout":True, "figHeightPerRow":4, "figWidthPerCol":6, "nColsGrid":4}
RECT_MULTI_PLOTTER = ppl.RectMultiPlotter(**_currKwargs)


**Import data**

In [3]:
RAW_DF = pd.read_csv(TRAIN_PATH)

In [4]:
RAW_DF.tail(3)

,id,date,store_nbr,family,sales,onpromotion
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


**Add some features via imported pipelines**

In [5]:
_uniqueFams = RAW_DF["family"].unique()

In [6]:
_lagPromPipe = preProcPipeHelp.AddLagFeats(ON_PROM_LAGS, targCol="onpromotion")

In [7]:
_maPromPipe = preProcPipeHelp.AddLeftMovingAverage(ON_PROM_WINDOW_VALS, [0], targCol="onpromotion")
_maSalePipe = preProcPipeHelp.AddLeftMovingAverage(SALE_WINDOW_VALS, [16], targCol="sales")

In [8]:
_removeNDaysBehindPipe = preProcPipeHelp.RemoveDatesNDaysBehindMax(N_DAYS_USE)

_pipeComps = [ ("Add lag features", preProcPipeHelp.AddLagFeats(SALE_LAG_VALS)),
               ("Add lag onpromotion feature", _lagPromPipe ),
               ("Add transactions data", preProcPipeHelp.AddNumbTransactionsData() ),
               ("Add store wide sums", preProcPipeHelp.AddStoreWideSums() ),
               ("Add moving average sale values", _maSalePipe ),
               ("Add moving average promotion values", _maPromPipe),
               ("Remove earlier data", _removeNDaysBehindPipe ),
               ("Add fractional number of promotions", preProcPipeHelp.AddFractProm() ),
               ("Add oil prices", preProcPipeHelp.AddOilPriceData() ),
               ("Add basic store info", preProcPipeHelp.AddStoreInfoData()),
               ("Add ordinal encoding for store type", preProcPipeHelp.OrdEncodeStoreType() ),
               ("Add ordinal encoding for store state", preProcPipeHelp.StoreStateOrdEncode() ),
               ("Add ordinal encoding for store city", preProcPipeHelp.StoreCityOrdEncode() ),
               ("Encode family ordinally", preProcPipeHelp.EncodeFamilyArbitrary(_uniqueFams)  ),
               ("Target encode combination of store-nbr AND family", preProcPipeHelp.TargEncodeFamilyStore() ),
               ("Add day of week", preProcPipeHelp.AddDayOfWeekFeat() ),
               ("Add day of month", preProcPipeHelp.AddDayOfMonth() ),
               ("Add day of year sin/cos", preProcPipeHelp.AddDayOfYearSinCos() )
             ]

preProcPipe = sk.pipeline.Pipeline(_pipeComps)

**Apply the pipeline preprocessing**

In [9]:
%%prun
#          369375860 function calls (368413681 primitive calls) in 176.059 seconds
PROC_FRAME = preProcPipe.fit_transform(RAW_DF)

/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:134: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summedFrame = useX.groupby(["date","store_nbr"]).sum().reset_index()
/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:103: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  meanDict = transFrame.groupby(["store_nbr"]).mean()["transactions"].to_dict()
/home/richard/Work/random/learning_ml/kaggle_comps/store_sales_github/shared_code/preproc_pipes.py:134: FutureWarning: The default value of numeric_only in DataFrameGro

**Calculate simple mutual information**

In [10]:
_onPromLags = ["onpromotion_l{}".format( int(x) ) for x in ON_PROM_LAGS]
_saleMaFeats = [ "sales_l16_ma{}".format(int(x)) for x in SALE_WINDOW_VALS]
_onPromMaFeats = ["onpromotion_l0_ma{}".format(int(x)) for x in ON_PROM_WINDOW_VALS]

In [11]:
cateVarKeys = ["store_nbr", "family_enc", "onpromotion", "day_of_week", "day_of_month", "store_type",
               "store_type_ordE", "store_cluster", "store_state_ordE", "store_city_ordE", "store_promotions"]

cateVarKeys += _onPromLags

#
useX = ["store_nbr", "family_enc", "onpromotion","day_of_week", "day_of_month",
        "sin_day_of_year", "cos_day_of_year", "fam_store_mean_enc","oil_price_w10",
        "store_type_ordE", "store_cluster", "store_state_ordE", "store_city_ordE",
        "store_promotions", "fract_promotions", "transactions"]


useX += ["sales_l16","sales_l17","sales_l18","sales_l19"]
useX += _onPromLags
useX += _saleMaFeats
useX += _onPromMaFeats

discFeats = [True if x in cateVarKeys else False for x in useX]


In [12]:
pd.set_option("display.max_columns",80)
PROC_FRAME.tail(3)

,id,date,store_nbr,family,sales,onpromotion,sales_l16,sales_l17,sales_l18,sales_l19,sales_l20,sales_l21,sales_l22,sales_l23,sales_l24,sales_l25,sales_l26,sales_l27,sales_l28,sales_l29,onpromotion_l1,onpromotion_l2,onpromotion_l3,onpromotion_l4,onpromotion_l5,onpromotion_l6,onpromotion_l7,onpromotion_l14,onpromotion_l15,onpromotion_l16,onpromotion_l17,onpromotion_l18,transactions,store_sales,store_promotions,store_sales_y,store_sales_y,sales_l16_ma2,sales_l16_ma3,sales_l16_ma4,sales_l16_ma5,sales_l16_ma6,sales_l16_ma7,sales_l16_ma14,sales_l16_ma21,onpromotion_l0_ma2,onpromotion_l0_ma3,onpromotion_l0_ma4,onpromotion_l0_ma5,onpromotion_l0_ma6,onpromotion_l0_ma7,onpromotion_l0_ma14,onpromotion_l0_ma21,fract_promotions,oil_price_w10,store_city,store_state,store_type,store_cluster,store_type_ordE,store_state_ordE,store_city_ordE,family_enc,fam_store_mean_enc,day_of_week,day_of_month,sin_day_of_year,cos_day_of_year
712797,2997191.0,2017-08-13,54.0,SEAFOOD,2.0,0.0,4.0,2.0,3.0,7.0,1.0,5.0,3.0,0.0,3.0,0.0,6.0,3.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1054.0,14246.827996,173.0,14246.827996,470145.323868,3.0,3.000000,4.00,3.4,3.666667,3.571429,3.142857,3.476190,0.5,0.333333,0.25,0.2,0.166667,0.142857,0.142857,0.285714,0.0,49.158333,El Carmen,Manabi,C,3,2,15,21,32,1.9225,6,13,-0.666089,-0.745872
712798,2998973.0,2017-08-14,54.0,SEAFOOD,0.0,0.0,4.0,4.0,2.0,3.0,7.0,1.0,5.0,3.0,0.0,3.0,0.0,6.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,818.0,11882.994000,126.0,11882.994000,392138.802000,4.0,3.333333,3.25,4.0,3.500000,3.714286,3.285714,3.238095,0.0,0.333333,0.25,0.2,0.166667,0.142857,0.142857,0.238095,0.0,48.828333,El Carmen,Manabi,C,3,2,15,21,32,1.9225,0,14,-0.678820,-0.734304
712799,3000755.0,2017-08-15,54.0,SEAFOOD,3.0,0.0,4.0,4.0,4.0,2.0,3.0,7.0,1.0,5.0,3.0,0.0,3.0,0.0,6.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,802.0,12666.858000,204.0,12666.858000,418006.314000,4.0,4.000000,3.50,3.4,4.000000,3.571429,3.214286,3.142857,0.0,0.000000,0.25,0.2,0.166667,0.142857,0.142857,0.238095,0.0,48.648571,El Carmen,Manabi,C,3,2,15,21,32,1.9225,1,15,-0.691351,-0.722519


In [13]:
%%prun
#          3069114 function calls (3049317 primitive calls) in 134.830 seconds
_currArgs = [ PROC_FRAME[useX] , PROC_FRAME["sales"].to_numpy() ]
mutualInfoVals = sk.feature_selection.mutual_info_regression( *_currArgs, discrete_features=discFeats)

In [14]:
miList = sorted([ [key,val] for key,val in zip(useX,mutualInfoVals) ], key=lambda x:x[1], reverse=True )
miList

[['fam_store_mean_enc', 1.707928383117947],
 ['sales_l16_ma21', 1.590653560162977],
 ['sales_l16_ma14', 1.586010375908784],
 ['sales_l16_ma7', 1.5658121466489963],
 ['sales_l16_ma6', 1.5496371294074693],
 ['sales_l16_ma5', 1.4909435414453922],
 ['sales_l16_ma4', 1.447932062392181],
 ['sales_l16_ma3', 1.4142995076291145],
 ['sales_l16_ma2', 1.3860623109220898],
 ['sales_l16', 1.324028395932202],
 ['sales_l19', 1.3171811875577237],
 ['sales_l17', 1.2911174356371404],
 ['sales_l18', 1.2866195830604141],
 ['family_enc', 1.191531619496828],
 ['onpromotion_l0_ma21', 0.8209570519536777],
 ['onpromotion_l0_ma14', 0.8083164602008459],
 ['onpromotion_l0_ma7', 0.7922823024647601],
 ['onpromotion_l0_ma6', 0.7430891806103785],
 ['onpromotion_l0_ma5', 0.6954681704320533],
 ['onpromotion_l0_ma4', 0.6459537457306865],
 ['onpromotion_l0_ma3', 0.5967193725715259],
 ['onpromotion_l0_ma2', 0.5488625591385432],
 ['onpromotion', 0.513379416755301],
 ['onpromotion_l7', 0.4822478059550548],
 ['fract_promotion

**Pragmatic estimation of mutual information score for variances from MEAN sales at each store/family combo**

In [15]:
PROC_FRAME["delta_sales"] = PROC_FRAME["sales"] - PROC_FRAME["fam_store_mean_enc"]

In [16]:
_currArgs = [ PROC_FRAME[useX] , PROC_FRAME["delta_sales"].to_numpy() ]
miDeltaSaleVals = sk.feature_selection.mutual_info_regression( *_currArgs, discrete_features=discFeats)

In [17]:
miListDelta = sorted([ [key,val] for key,val in zip(useX,miDeltaSaleVals) ], key=lambda x:x[1], reverse=True )
miListDelta

[['fam_store_mean_enc', 4.2664586332001075],
 ['family_enc', 1.9969299779788448],
 ['store_nbr', 1.8586630433253557],
 ['store_cluster', 1.1451038102470132],
 ['store_city_ordE', 1.1075726706671931],
 ['sales_l16_ma21', 1.0916607209622011],
 ['sales_l16_ma14', 1.0762697128638061],
 ['sales_l16_ma7', 1.0510084240080362],
 ['sales_l16_ma6', 1.0422953809788291],
 ['sales_l16_ma5', 1.0310966773412256],
 ['sales_l16_ma4', 1.0152350404923354],
 ['sales_l16_ma3', 0.9939196080605521],
 ['sales_l16_ma2', 0.9534171094492843],
 ['store_state_ordE', 0.9459210891732499],
 ['sales_l18', 0.8985876652134568],
 ['sales_l17', 0.8933146519728181],
 ['sales_l19', 0.8896817298666289],
 ['sales_l16', 0.8852925387961124],
 ['onpromotion_l0_ma21', 0.6788170484639906],
 ['onpromotion_l0_ma14', 0.6625715822052136],
 ['onpromotion_l0_ma7', 0.6367512721402369],
 ['onpromotion_l0_ma6', 0.5902101178757926],
 ['store_type_ordE', 0.584028254607897],
 ['onpromotion_l0_ma5', 0.5521210895085673],
 ['onpromotion_l0_ma4',

**Look at the naive forecast average score as an interesting baseline**

In [18]:
def getScoreForNaive(inpFrame, predCol="sales_l16"):
    useFrame = inpFrame.copy()
    addSqrLogErrorToFrame(useFrame, predCol=predCol)
    return math.sqrt(useFrame["sqr_log_error"].mean())

def addSqrLogErrorToFrame(inpFrame, predCol="pred_A", actCol="sales", outCol="sqr_log_error"):
    inpFrame[outCol] = inpFrame.apply( lambda x: (math.log(x[predCol]+1) - math.log(x[actCol]+1))**2 ,axis=1)


In [19]:
getScoreForNaive(PROC_FRAME)

NameError: name 'math' is not defined

In [ ]:
getScoreForNaive(PROC_FRAME,predCol="fam_store_mean_enc")